In [25]:
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier , DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier , AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import BayesianRidge,LinearRegression,SGDClassifier,LogisticRegression
from sklearn.neighbors.nearest_centroid import NearestCentroid

In [26]:
def acuaracy(cm):
    diagonal_index= 0
    total=0
    diagonal_total=0
    for x in cm:
        index=0
        for y in x:
            if(diagonal_index==index):
                diagonal_total=diagonal_total+y
            total=total+y
            index=index+1
        diagonal_index=diagonal_index+1
    acuaracy =  diagonal_total/total
    return acuaracy

In [27]:
def boostrap(modelos,modelo_strg,maxima_iteracion,tamanio_muestra,predictors,targets,labels):
    Datos=[]
    string=["modelo","promedio","minimo","maximo"]
    Datos.append(string)
    index=0;
    for clf in modelos:
        minimo =1;
        maximo=-1;
        promedio=0;
        temporal=[]
        for x in range(0,maxima_iteracion):
            pred_train, pred_test, tar_train, tar_test = train_test_split(predictors, targets, test_size=tamanio_muestra)
            temporal_y = tar_train.values
            temporal_x = pred_train.values
            clf = clf.fit(temporal_x, temporal_y)
            y_true = tar_test.values
            y_pred = clf.predict(pred_test)
            try:
                cnf_matrix = confusion_matrix(y_true, y_pred, labels=labels)
            except:
                y_pred2=[]
                for x in y_pred:
                    y_pred2.append(round(x))
                cnf_matrix = confusion_matrix(y_true, y_pred2, labels=labels)
            dato= acuaracy(cnf_matrix)
            promedio=promedio+dato
            if(dato<minimo):
                minimo=dato
            if(dato>maximo):
                maximo=dato
        temporal.append(modelo_strg[index])        
        temporal.append(promedio/maxima_iteracion)
        temporal.append(minimo)
        temporal.append(maximo)
        Datos.append(temporal)
        index=index+1
    return Datos 

<hr>

In [28]:
f = open('../final.csv','r')
final = pd.read_csv(f)
final.drop('Unnamed: 0', axis=1, inplace=True)
final.drop('Sales', axis=1, inplace=True)

In [29]:
predictors = final[["StoreType","Assortment","CompetitionDistance_int","Customers","Open_int"]]
targets = final.Sales_rango 
labels=[0,1,2,3]

<hr>

In [30]:
#modelos
AC = DecisionTreeClassifier(max_depth=5) 
AR = DecisionTreeRegressor(max_depth=5)
RF = RandomForestClassifier(n_estimators=10,max_depth=5)
RN = MLPClassifier(hidden_layer_sizes=(5,), max_iter=10)
GN = GaussianNB()
LN = LinearRegression()
BR = BayesianRidge()
SGD= SGDClassifier(loss="hinge", penalty="l2")
NC = NearestCentroid()
ADA = AdaBoostClassifier( n_estimators=10, learning_rate=1.5, random_state=1)
GBRT = GradientBoostingClassifier(random_state=1, n_estimators=10,max_depth=5, learning_rate=0.01)
RLOG = LogisticRegression()

modelos=[AC,AR,RF,RN,GN,LN,BR,SGD,NC,ADA,GBRT,RLOG]
modelos_string=["AC","AR","RF","RN","GN","LN","BR","SGD","NC","ADA","GBRT","RLOG"]

In [31]:
boostrap(modelos,modelos_string,5,.1,predictors,targets,labels)

/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


[['modelo', 'promedio', 'minimo', 'maximo'],
 ['AC', 0.72537627431897056, 0.72337078872602512, 0.72746040640575693],
 ['AR', 0.73506748852252723, 0.73007540232597001, 0.74075166386488533],
 ['RF', 0.71476489613747418, 0.70638314605636987, 0.72261381622280552],
 ['RN', 0.50058100097325042, 0.3557377532662872, 0.59700553474700402],
 ['GN', 0.65120476597752686, 0.64950206938586919, 0.65219571179992331],
 ['LN', 0.59829961945591903, 0.59703466239230618, 0.59990380375966967],
 ['BR', 0.59798178709994865, 0.59487867382077364, 0.60009012166424669],
 ['SGD', 0.31676448324338136, 0.29162119916241486, 0.34642797455785923],
 ['NC', 0.36133738362776613, 0.34342957698017124, 0.38314605636987448],
 ['ADA', 0.70076778639612269, 0.69790898634500254, 0.7023230208118284],
 ['GBRT', 0.73480992125519806, 0.73209071873064557, 0.73766478898162624]]